In [ ]:
from readurl import *
from analysis import *

# load data from http://www.worldometers.info/coronavirus/country/
germany = get_data('germany')
italy = get_data('italy')
france = get_data('france')
china = get_data('china') 
us = get_data('us')            # avail: total, active

# load data from https://datawrapper.dwcdn.net/fIw01/56/
berlin = get_data_berlin()     # avail: total,  in hospital, severe, death 

In [ ]:
country = germany           # choose country
start_date = "Mar01"        # analysis starting on  use format Apr01
x0 = 5                      # increase during last x0 days
n_fit = 7                   # make fit over last week

countries = [china, germany, italy, us]
dN_N(countries, start_date)                   # change over total 
fitting = Analysis(country=country, start_date=start_date, n_fit=n_fit)
fitting.fit_total('exp')
fitting.fit_total('log')
fitting.daily_incr()
fitting.growth_rate()
fitting.plotactive()

In [ ]:
# Local data for Berlin 

fitting = Analysis(country=berlin, start_date=start_date, n_fit=n_fit)
fitting.fit_total('exp')
fitting.fit_total('log')
fitting.daily_incr()
fitting.growth_rate()


In [ ]:
# Another formular for logistic function, K: max of the curve; r: growth rate;
Y_num = Data_DE[start_ind:]

#r_CN=0.235
def fun_logt(t,K,r):
    #r = r_CN
    P0 = Y_num[5]
    y = K/(1+((K-P0)/P0)*np.exp(-r*t))
    return y
popt, pcov = curve_fit(fun_logt, X_date, Y_num, p0=(60000,0.25))

xx = np.linspace(0, 55, 55)
yy = fun_logt(xx, *popt)
plt.plot(X_date,Y_num,'o',label='DE_total')
plt.plot(xx, fun_logt(xx, *popt),'--',lw=2,label='logt_fit: r=%.2f'%popt[1])

plt.title('Germany,next day: %.0f'%(fun_logt(X_date[-1]+1,*popt)),fontsize=14)
plt.xlabel('Day (+ March 3rd)',fontsize=13)
plt.ylabel('# of cases.',fontsize=13)
plt.tick_params(labelsize=13,length=5)
plt.legend(fontsize=12,loc='upper left')
plt.grid()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# China, CD: Clinically diagnosed
Y_num_noCD = [62,121,198,291,440,571,830,1287,1975,2744,4515,5974,7711,9692,11791,14380,17205,20438,24324,28018,31161,34546,40171,42638,44653,46472,48467,49970,51091]
Y_num_CD = [62,121,198,291,440,571,830,1287,1975,2744,4515,5974,7711,9692,11791,14380,17205,20438,24324,28018,31161,34546,40171,48315,55220,58761,63851,66492,68500,70548,72436,74185,75002,75891,76288,76936,77150,77658,78064,78497,78824,79251,80026,80151,80270,80409,80552,80651,80695,80735,80754,80778,80793,80813,80824,80844]
Y_num = Y_num_CD
X_date = [] # data from Jan 17st
for i in range(len(Y_num)):
    X_date.append(i+1)

# logistic function, K: max of the curve; r: growth rate;
def fun_logt(t,r):
    K = Y_num[-1]
    P0 = Y_num[0]
    y = K/(1+((K-P0)/P0)*np.exp(-r*t))
    return y
popt, pcov = curve_fit(fun_logt, X_date, Y_num, p0=(0.1))
r_CN = popt[0]

# logistic function, L: max of the curve; k: growth rate; x0: midpoint
def fun_logt(x,k,x0):
    L = max(Y_num)
    y = L/(1+np.exp(-k*(x-x0)))
    return y
popt, pcov = curve_fit(fun_logt, X_date, Y_num, p0=(0.2,20))
k_CN = popt[0]
print('k_CN=',k_CN,'x0=',popt[1])


xx = np.linspace(1, 59, 50)
yy = fun_logt(xx, *popt)
plt.plot(X_date,Y_num,'o',label='CN_tot')
plt.plot(xx,yy,'--',lw=3,label='logt_fit: k=%5.3f' % popt[0])

# exponential fit
Y_num_CD = [62,121,198,291,440,571,830,1287,1975,2744,4515,5974,7711,9692,11791,14380]
Y_num = Y_num_CD
X_date = [] # data from Jan 17st
for i in range(len(Y_num)):
    X_date.append(i+1)
def fun_exp(x, a, b):
    return a*np.exp(b*x)
popt, pcov = curve_fit(fun_exp, X_date, Y_num, p0=(60,0.2))
xx = np.linspace(1, 22, 50)
yy = fun_exp(xx, *popt)
plt.plot(xx,yy,'-.',lw=2,label='exp_fit: b=%5.3f' % popt[1])

#plt.ylim(0,8000)
plt.title('China (incl C.D.), Jan.17th - Mar.14th',fontsize=14)
plt.xlabel('Day (+Jan.17th)',fontsize=13)
plt.ylabel('# of cases.',fontsize=13)
plt.tick_params(labelsize=13,length=5)
plt.legend(fontsize=12)
plt.grid()
plt.show()
popt[0]